# Supramolecular cage analysis

In this Jupyter notebook, we will perform a supramolecular cage analysis of a host-guest pair (CSD Identifier: [718469](https://www.ccdc.cam.ac.uk/structures/Search?Ccdcid=718469)) consisting of HOMRUX (the host) and BnNMe3 (the guest). The original X-ray diffraction (XRD) structure of the complex was obtained from the Cambridge Structural Database (CSD), and any non-cage atoms and fragments were removed using Diamond Crystal and Molecular Structure Visualization software.

Supramolecular cages have important applications in areas such as drug delivery, catalysis, and materials science, and understanding their properties and behavior is of great interest. One important aspect of analyzing these systems is the identification of cavities within the host molecule, which can play a key role in determining the guest encapsulation behavior. 

To this end, we will use pyKVFinder, a software package that employs a geometric algorithm, to identify cavities in molecular structures and calculate the van der Waals volume of the guest molecule.

To assess the performance of any cavity detection software, the calculated volumes must be compared to the “real” cavity volume, a value that cannot be determined by experimental methods. So, we are taking advantage of Rebek’s rule of thumb: in any (biological and artificial) host-guest system, the guest occupies 55% of the available space within the cavity, when guest encapsulation is only driven by weak interactions (e.g., dipole-dipole, dipole-induced dipole, London dispersion forces).

## Requirements

In [1]:
# Import required modules
import os
import nglview
import zipfile
import pyKVFinder

In [2]:
# Unzip required files
if not os.path.isdir('./data'):
    with zipfile.ZipFile('./data.zip', 'r') as zipped:
        zipped.extractall('./data')

## Host's cavity detection and volume characterization

In [3]:
# Cavity detection parameters
step = 0.25
probe_out = 10.0
removal_distance = 1.75
volume_cutoff = 5.0

In [4]:
# Host filepath
host = './data/HOMRUX.pdb'

# Atomic information
atomic = pyKVFinder.read_pdb(host)

# Grid dimensions
vertices = pyKVFinder.get_vertices(atomic, probe_out=probe_out, step=step)

# Cavity detection
ncav, cavities = pyKVFinder.detect(
    atomic,
    vertices,
    step=step,
    probe_out=probe_out,
    removal_distance=removal_distance,
    volume_cutoff=volume_cutoff,
)

# Spatial characterization
surface, cavity_volume, area = pyKVFinder.spatial(cavities, step=step)

# Depth characteriation
depths, max_depth, avg_depth = pyKVFinder.depth(cavities, step=step)

# Export cavity
pyKVFinder.export('./results/host-cavity.pdb', cavities, surface, vertices, step=step, B=depths)

# Show volume
print(f"Cavity volume: {cavity_volume['KAA']} Å³")

Cavity volume: 283.14 Å³


In [5]:
# Visualize the host cavity

# Create NGL Viewer widget
view = nglview.NGLWidget()

# Disply referece 6W02 as grey cartoon
pdb = view.add_component(host)
pdb.clear()
pdb.add_licorice(color='grey')

# Display surface points colored by hydrophobicity
cavs = view.add_component('./results/host-cavity.pdb')
cavs.clear()
# cavs.add_point(colorScheme='bfactor', colorScale='rainbow', colorReverse=True)
cavs.add_surface(colorScheme='bfactor', colorScale='rainbow', colorReverse=True, surfaceType='vws', probeRadius=0.125)

# Display on NGL viewer
view

NGLWidget()

## Guest's van der Waals volume

In [6]:
# Guest modelling parameters
step = 0.1
padding = 3
radii = {'GEN': {"C": 1.7, "H": 1.2, "N": 1.55, "O": 1.52, "B":1.85, "F":1.47, "CL": 1.75, "CO":1.8}}

In [7]:
# Guest filepath
guest = './data/BnNMe3.pdb'

# Create Molecule
molecule = pyKVFinder.Molecule(guest, radii=radii)

# Create vdW surface representation
molecule.vdw(step=step, padding=padding)

# Calculate vdW volume
guest_volume = molecule.volume()

# Export cavity
molecule.export('./results/guest-vdw-volume.pdb')

# Show volume
print(f"Guest vdW volume: {guest_volume} Å³")

Guest vdW volume: 154.51 Å³


## Compare cavity and guest vdw volumes using Rebek's rule

In [8]:
# Relative error according to expected volume
expected = guest_volume / 0.55
calculated = cavity_volume['KAA']
RE = (calculated - expected) / expected

print(f"Relative Error (%): {RE * 100:.1f}")

Relative Error (%): 0.8
